Import the required libraries

In [325]:
import nsepy
import pandas as pd
import numpy as np
from datetime import date,timedelta

Pull the data for the last 360 days, we would be checking the momentum for each stock in last X days.

Refer to NSEPy package , which has good documentation to extract historical data.

https://nsepy.xyz/

In [327]:
# Set the dates and time ranges
today = date.today()

# today_1 = today - timedelta(days = 1)
# bf_15_days = today - timedelta(days = 15)
# bf_30_days = today - timedelta(days = 30)
# bf_60_days = today - timedelta(days = 60)
# bf_90_days = today - timedelta(days = 90)
# bf_180_days = today - timedelta(days = 180)
bf_360_days = today - timedelta(days = 360)

# print(today)
# print(bf_15_days)
# print(bf_30_days)
# print(bf_60_days)

# Calling the nsepy functions
data = nsepy.get_history(symbol="INFY", start=bf_360_days, end=today)

# print(data)

Extract the list of stocks for which you need to pull the price

In [181]:
nf150 = pd.read_csv('NIFTY MIDCAP 150.csv')
nf150['Symbol'].values

array(['WHIRLPOOL', 'SYMPHONY', 'DHANI', 'ABFRL', 'ENDURANCE', 'NIACL',
       'GSPL', 'OIL', 'SRTRANSFIN', 'TTKPRESTIG', 'BAYERCROP',
       'GUJGASLTD', 'RELAXO', 'THERMAX', 'LALPATHLAB', 'AJANTPHARM',
       'FRETAIL', 'SFL', 'CESC', 'HONAUT', 'HAL', 'NHPC', 'ITI', 'SAIL',
       'MRPL', 'VINATIORGA', 'POLYCAB', 'TATACOMM', 'ADANIENT', 'RECLTD',
       'VBL', 'COROMANDEL', 'FEDERALBNK', 'NATCOPHARM', 'SUNDARMFIN',
       'EMAMILTD', 'SANOFI', 'LTTS', 'LICHSGFIN', 'JMFINANCIL', 'APLLTD',
       'CROMPTON', 'AKZOINDIA', 'GODREJIND', 'SOLARINDS', 'NAVINFLUOR',
       'ABB', 'L&TFH', 'SKFINDIA', 'SCHAEFFLER', 'MINDAIND', 'COFORGE',
       'CASTROLIND', 'UNIONBANK', 'PHOENIXLTD', 'ERIS', 'GLENMARK',
       'ASTRAZEN', 'PFIZER', 'PRESTIGE', 'SUMICHEM', 'SJVN', 'IBULHSGFIN',
       '3MINDIA', 'NAM-INDIA', 'FORTIS', 'BANKINDIA', 'RAJESHEXPO',
       'ZYDUSWELL', 'GLAXO', 'AARTIIND', 'PIIND', 'TATAPOWER', 'TATACHEM',
       'VGUARD', 'MRF', 'TRENT', 'IPCALAB', 'AIAENG', 'GILLETTE', 'AAVAS',


For some symbols the nsepy function was not returning the data and hence it was breaking. Handle the exception scenarios

In [319]:
symbols = nf150['Symbol'].values
# print(symbols)
exception_symbols = ['BBTC','M&MFIN']
index =[]
for i in exception_symbols:
    if np.where(symbols == i)[0].size >0:
        index.append(np.where(symbols == i))
# index = np.where(symbols == exception_symbols)
print(index)
symbols= np.delete(symbols,index)
# print(symbols)
print(len(symbols))

[(array([118], dtype=int64),), (array([136], dtype=int64),)]
148


Create new DataFrame to track the required columns which include:
* Ticker - Ticker Name
* Current price
* Price at the end of previous 7th Session
* Price at the end of previous 15th Session
* Price at the end of previous 30th Session
* Price at the end of previous 60th Session
* Price at the end of previous 90th Session
* Price at the end of previous 180th Session
* Price at the end of previous 365th Session (or till whenever historical data available)

Note: No of sessions can be approximated to number of days i.e previous 7th session can also be approximated to price before 7 days

In [320]:
new_columns =[
        'Ticker'
        ,'Current Price'
        ,'Price BF 7 Days'
        ,'Price BF 15 Days'
        ,'Price BF 30 Days'
        ,'Price BF 60 Days'
        ,'Price BF 90 Days'
        ,'Price BF 180 Days'
        ,'Price BF 360 Days'
]

df1 = pd.DataFrame(columns=new_columns)
# df1

In [328]:
# function to convert the Pandas timestamp to Date
def price_asof(date1):
    return date1.to_pydatetime().date() 

Below logic, loops through the list of symbols and fetches the prices as mentioned above and creates a new dataframe

In [321]:
for symbol in symbols:
    data = nsepy.get_history(symbol=f'{symbol}', start=bf_360_days, end=today)
    
    data_df = pd.DataFrame(data)
    data_df.reset_index(inplace=True)
    
    data_df['Date']=pd.to_datetime(data_df['Date'])
    
    data_df['Date']=data_df['Date'].apply(lambda x: price_asof(x))
    
    data_df.sort_values('Date',ascending=False,inplace=True)
    data_df.reset_index(inplace=True)
    data_df.drop('index',axis=1,inplace=True) 
    
    fetch_index = [0,7,15,30,60,90,180,len(data_df.index)-1]
    
    df1= df1.append(pd.Series
               (
                   [
                    data_df['Symbol'].unique()[0]
                    ,data_df.loc[fetch_index[0],'Close']
                    ,data_df.loc[fetch_index[1],'Close']
                    ,data_df.loc[fetch_index[2],'Close']
                    ,data_df.loc[fetch_index[3],'Close']
                    ,data_df.loc[fetch_index[4],'Close']
                    ,data_df.loc[fetch_index[5],'Close']
                    ,data_df.loc[fetch_index[6],'Close']
                    ,data_df.loc[fetch_index[7],'Close']
                   ]

              , index = new_columns     
            ),
               ignore_index=True
              )    
    
df1    

,Ticker,Current Price,Price BF 7 Days,Price BF 15 Days,Price BF 30 Days,Price BF 60 Days,Price BF 90 Days,Price BF 180 Days,Price BF 360 Days
0,WHIRLPOOL,2528.00,2159.40,2148.55,2129.10,2202.80,2022.60,1815.75,2373.50
1,SYMPHONY,984.45,924.75,851.50,824.10,945.65,840.65,870.80,1162.45
2,DHANI,322.10,287.55,290.20,195.75,242.90,166.85,102.40,214.35
3,ABFRL,164.10,160.20,163.70,156.50,132.55,136.35,149.00,231.70
4,ENDURANCE,1240.60,1160.65,1188.20,1057.05,1109.35,1021.50,592.35,1081.30
...,...,...,...,...,...,...,...,...,...
143,TORNTPOWER,318.75,335.40,324.45,310.65,316.75,344.60,291.85,284.00
144,INDHOTEL,121.35,129.10,127.80,108.50,95.50,97.25,75.05,145.00
145,BHARATFORG,526.60,553.75,539.40,499.75,449.35,499.55,231.55,483.15
146,OBEROIRLTY,533.20,512.05,530.55,429.45,399.95,364.50,306.40,530.65


Create new variables to track the percentage difference between different time periods

In [322]:
df1['7 Days Perc Return'] = (df1['Current Price']-df1['Price BF 7 Days'])/df1['Price BF 7 Days']
df1['15 Days Perc Return'] = (df1['Current Price']-df1['Price BF 15 Days'])/df1['Price BF 15 Days']
df1['30 Days Perc Return'] = (df1['Current Price']-df1['Price BF 30 Days'])/df1['Price BF 30 Days']
df1['60 Days Perc Return'] = (df1['Current Price']-df1['Price BF 60 Days'])/df1['Price BF 60 Days']
df1['90 Days Perc Return'] = (df1['Current Price']-df1['Price BF 90 Days'])/df1['Price BF 90 Days']
df1['180 Days Perc Return'] = (df1['Current Price']-df1['Price BF 180 Days'])/df1['Price BF 180 Days']
df1['360 Days Perc Return'] = (df1['Current Price']-df1['Price BF 360 Days'])/df1['Price BF 360 Days']

Values are sorted based on the descending order of last 15 days percentage difference. These scripts are in good momentum and should be tracked further to understand entry zones.

In [324]:
df1.sort_values('15 Days Perc Return',ascending=False,inplace=True)
df1[:20]

,Ticker,Current Price,Price BF 7 Days,Price BF 15 Days,Price BF 30 Days,Price BF 60 Days,Price BF 90 Days,Price BF 180 Days,Price BF 360 Days,7 Days Perc Return,15 Days Perc Return,30 Days Perc Return,60 Days Perc Return,90 Days Perc Return,180 Days Perc Return,360 Days Perc Return
37,LTTS,2348.65,1830.50,1772.70,1651.65,1629.80,1588.20,1104.50,1468.45,0.283065,0.324900,0.422002,0.441066,0.478812,1.126437,0.599408
19,HONAUT,38250.35,31904.35,30991.25,29063.50,32671.10,30649.95,26870.40,27315.05,0.198907,0.234231,0.316096,0.170770,0.247974,0.423512,0.400340
101,PAGEIND,27478.50,24631.00,23231.15,21729.20,21494.20,19101.40,16757.00,23393.10,0.115606,0.182830,0.264589,0.278415,0.438559,0.639822,0.174641
0,WHIRLPOOL,2528.00,2159.40,2148.55,2129.10,2202.80,2022.60,1815.75,2373.50,0.170696,0.176607,0.187356,0.147630,0.249876,0.392262,0.065094
1,SYMPHONY,984.45,924.75,851.50,824.10,945.65,840.65,870.80,1162.45,0.064558,0.156136,0.194576,0.041030,0.171058,0.130512,-0.153125
121,GODREJPROP,1364.10,1291.70,1182.85,991.40,879.15,891.90,687.15,988.30,0.056050,0.153232,0.375933,0.551612,0.529432,0.985156,0.380249
133,MPHASIS,1554.55,1324.80,1353.30,1310.10,1317.60,1197.50,673.40,922.25,0.173422,0.148711,0.186589,0.179835,0.298163,1.308509,0.685606
87,YESBANK,17.55,18.30,15.45,14.15,13.35,15.05,24.75,46.95,-0.040984,0.135922,0.240283,0.314607,0.166113,-0.290909,-0.626198
55,ERIS,595.15,539.85,528.40,506.20,530.75,525.35,403.95,474.00,0.102436,0.126325,0.175721,0.121338,0.132864,0.473326,0.255591
5,NIACL,132.25,129.50,118.30,104.55,105.45,117.80,106.40,135.45,0.021236,0.117921,0.264945,0.254149,0.122666,0.242951,-0.023625
